# COGS 108 - Final Project (Group 7)

# Overview

*Fill in your overview here*

# Names and Members IDs

- Purisa Jasmine Simmons - A12802561
- Eryue Xu - A15122722
- Bar Yosef - A13368857
- Jing Lang - A14906233
- Sao I Chan - A15720032
- Jasmine Palafox - A14052116


# Research Question

The main question that we would like to explore is:
What is the relationship between an adult population’s awareness of climate change and
the amount of greenhouse gases released by that population’s region?
In order to develop an answer this question, we will also explore the following sub-questions:
- How does public awareness of climate change vary by state within the U.S.?
- How have greenhouse gas emissions in the United States changed over time, using data
collected from the following years: 2014, 2016, 2018?
- Does a state’s overall political lean (more Democrat or Republican) affect that state’s adult
population’s concern for climate change?
- On a global scale, how do greenhouse gas emissions vary by country? Is there a correlation
between a country’s greenhouse gas emissions and their awareness of climate change?

## Background and Prior Work

*Fill in your background and prior work here* 

References (include links):
- 1)
- 2)

# Hypothesis


We hypothesize that regions with a greater awareness of climate change will produce lower
amounts of greenhouse gases emissions, when compared to regions with lesser degrees of
climate change awareness, especially when the industrialization of the region under study is
controlled for. This hypothesis is based on the idea that a public awareness of climate change
would cause local governments to produce policy that addresses climate change by regulating
greenhouse gas emissions (Bord, O’Connor, & Fisher, 2000).

# Dataset(s)

We have nine datasets that we are using in our final project. We will analyze the data contained in these nine datasets in order to guide the analysis of our research questions.

Dataset 1
- Dataset Name: YPCCC_2014_State.csv
- Link to the dataset: https://climatecommunication.yale.edu/visualizations-data/ycom/
- Number of observations: 51 states

Dataset 2

- Dataset Name: YCOM_2016.csv
- Link to the dataset: https://climatecommunication.yale.edu/visualizations-data/ycom-us-2016/?est=happening&type=value&geo=county
- Number of observations: 51 states

Dataset 3

- Dataset Name: YCOM_2018.csv
- Link to the dataset: https://climatecommunication.yale.edu/visualizations-data/ycom-us-2018/?est=happening&type=value&geo=county
- Number of observations: 51 states

These thress datasets come from the study on climate change awareness conducted by the Yale Program on Climate Change Communication. It gives us information related to a geographic variation in opinions of climate change, organized at the level of county and/or state including the following categories: beliefs, risk perceptions, policy support, behaviors, awareness, worried, etc.

Dataset 4

- Dataset Name: Carbon_dioxide_emission_by_state.xlsx
- Link to the dataset: https://www.eia.gov/environment/emissions/state/analysis/
- Number of observations: 51 states

This dataset contains the data related to the carbon dioxide emission level by state from 2005 to 2016. 

Dataset 5

- Dataset Name: State_Party_Affiliation_2014.csv
- Link to the dataset: https://www.pewforum.org/religious-landscape-study/compare/party-affiliation/by/state/
- Number of observations: 51 states * 4 columns = 204 observations

This dataset contains information related to the political party affiliation of each state in 2014 based on a subsample of each state's population. 

Dataset 6

- Dataset Name: 2014_State_Size_Population_Density.csv
- Link to the dataset: https://www.states101.com/populations
- Number of observations: 57 states * 3 columns = 171 observations

This dataset contains information for the population, land area in square miles, and population density (people per square mile) of each U.S. state and territory. 

Dataset 7

- Dataset Name: GreenhouseGasInventoryDataUS.csv
- Link to the dataset: https://cfpub.epa.gov/ghgdata/inventoryexplorer/
- Number of observations: We have 27 years data

This dataset contains the total amount of each category of gases in U.S. from 1990 to 2017 including carbon dioxide, methane, nitrous oxide, and fluorinated gases. 

Dataset 8

- Dataset Name: annual_co2_emissions_per_country.csv
- Link to the dataset: https://ourworldindata.org/co2-and-other-greenhouse-gas-emissions#per-capita-co2-emiss
- Number of observations: For each country, there are 265 years data is collected. 

This dataset describes the annual CO2 and other greenhouse gas emissions per country globally, starting from 1751 until 2016, measured in tonnes per year.

Dataset 9

- Dataset Name: awareness_by_percentage.xlsx
- Link to the dataset: https://www.nature.com/articles/nclimate2728
- Number of observations: 119 countries in the dataset

This dataset contains datas that can help us to analyze the relationship between different categories and people's level of awareness of climate change.

# Setup

In [ ]:
#import libraries that are needed
%matplotlib inline

import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import numpy as np

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (17, 5)
plt.rcParams.update({'font.size': 16})
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns

import shapely.geometry as shp

import sklearn.neighbors as skn
import sklearn.metrics as skm

pd.options.display.max_rows = 10

#improve resolution
#comment this line if erroring on your machine/screen
%config InlineBackend.figure_format ='retina'

In [ ]:
#Import Other Necessary Python Libraries:
import math
import operator
import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from numpy.polynomial.polynomial import polyfit

import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest

import os 
import earthpy as et

import folium
from folium import plugins

import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, Resampling

# Data Cleaning

Describe your data cleaning steps here.

#### Dataset 1: Read in data about population sentiment per state in 2014 (for Simmons Analysis) 

In [ ]:
#File names and datasets 
#YPCCC_2014_State.csv - Yale Study conducted in 2014 w/ population sentiments of climate change
df_YPCCC_2014_State = pd.read_csv("Datasets/YPCCC_2014_State.csv")

#Choose which columns from the original dataset to keep: 
columns_keep = ['Statecode', 'Statename', 'TotalPop', 'happening', 'human', 'worried', 'personal', 'CO2limits', 'regulate', 'fundrenewables']
df_YPCCC_2014_State_new = df_YPCCC_2014_State[columns_keep]

#Drop District of Columbia: 
df_YPCCC_2014_State_new = df_YPCCC_2014_State_new[df_YPCCC_2014_State_new.Statecode != "DC"]
df_YPCCC_2014_State_new

#### Dataset 2: Read in data about awareness of climate change in 2016

In [ ]:
#import dataset
df_YCOM_2016 = pd.read_csv("Datasets/YCOM_2016.csv")

#Specify which columns in 2016 data we would like to keep in the dataframe for our analysis
columns_2016_keep = ['GeoName', 'happening', 'human', 'worried', 'personal', 'CO2limits', 
                     'regulate', 'fundrenewables']
df_YCOM_2016_new = df_YCOM_2016[columns_2016_keep]
df_YCOM_2016_state = df_YCOM_2016_new.loc[0:51]
df_YCOM_2016_state = df_YCOM_2016_state.drop(index = 0)
df_YCOM_2016_state.dropna()
df_YCOM_2016_state = df_YCOM_2016_state[df_YCOM_2016_state.GeoName != "District of Columbia"]
#df_YCOM_2016_state

#### Dataset 3: Read in data about awareness of climate change in 2018

In [ ]:
#import dataset
df_YCOM_2018 = pd.read_csv("Datasets/YCOM_2018.csv", encoding='ISO-8859-1')

#### Dataset 4: Read in data about carbon dioxide emission per state (for Simmons Analysis)

In [ ]:
#Carbon_dioxide_emission_by_state.xlsx - Carbon Dioxide per state from years 2005-2016
df_CO2_state = pd.read_excel("Datasets/Carbon_dioxide_emission_by_state.xlsx")

#Drop NaN values and rename columns with proper years for 2012, 2014, and 2016 data
col_years = ['State']
for year in list(range(2005, 2017)):
    col_years.append(str(year))

col_years.append('Percent Change')
col_years.append('Absolute')

df_CO2_state.columns=col_years

#Deleting unnecessary rows from beginning, renaming index to be the state names:
new_df_CO2_state = df_CO2_state.iloc[4:55]
new_df_CO2_state = new_df_CO2_state.set_index('State')

#Right now, only concerned with year 2014:
df_CO2_state_2014 = pd.DataFrame(new_df_CO2_state[:]['2014'])
df_CO2_state_2014.columns=(["CO2_Emissions_in_2014"])


#Drop District of Columbia: 
df_CO2_state_2014 = df_CO2_state_2014.drop("District of Columbia", axis=0)
#df_CO2_state_2014

df_CO2_state2 = pd.read_excel("Datasets/Carbon_dioxide_emission_by_state.xlsx", header=4)
df_CO2_state2_dropped = df_CO2_state2.dropna()
## only keep the data from 2012, 2014, 2016
columns_CO2 = ['State',2012, 2014, 2016]
df_CO2_state_keep = df_CO2_state2_dropped[columns_CO2]
df_CO2_state_keep = df_CO2_state_keep.drop(51, axis = 0)
#Drop District of Columbia: 
df_CO2_state_keep = df_CO2_state_keep[df_CO2_state_keep.State != "District of Columbia"]
df_CO2_state_keep.columns = ['State', '2014', '2016', '2018']
#df_CO2_state_keep

#### Dataset 5: Read in data about state party affiliations for 2014 (for Simmons Analysis)

In [ ]:
df_party_state = pd.read_csv("Datasets/State_Party_Affiliation_2014.csv")
df_party_state = df_party_state.set_index("State")
df_party_state.columns=["Republican", "No_lean", "Democrat", "Sample_Size"]
df_party_state.Republican = df_party_state.Republican.apply(lambda x: x.replace('%',''))
df_party_state.No_lean = df_party_state.No_lean.apply(lambda x: x.replace('%',''))
df_party_state.Democrat = df_party_state.Democrat.apply(lambda x: x.replace('%',''))
df_party_state

#Drop the district of columbia since it isn't a state: 
df_party_state = df_party_state.drop("District of Columbia")
df_party_state




#### Dataset 7: Read in data about annual greenhouse gas in US

In [ ]:
#import greenhouse gas data
annual_gas_us = pd.read_csv('Datasets/GreenhouseGasInventoryDataUS.csv')

#Dataset in the different orientation
#Transpose the dataset and rename the column
annual_gas_us = annual_gas_us.set_index('Gas').rename_axis(['Year']).transpose().reset_index()
annual_gas_us.rename(columns = {'index' : 'Year'}, inplace = True)

#### Dataset 8: Read in data about annual CO2 emission globally (for Yolanda analysis)

In [ ]:
#import global CO2 emission data
df_country_emission = pd.read_csv('Datasets/annual_co2_emissions_per_country.csv')

#rename the column and reset the index
df_country_emission = df_country_emission.rename(index=str, columns={"Annual CO₂ emissions (Global Carbon Project (2017)) (tonnes)": "Annual CO₂ emissions(tonnes)"})
df_country_emission.set_index('Entity', inplace=True)

#drop any unavailable or no value data
df_country_emission= df_country_emission.dropna()
df_country_emission= df_country_emission[df_country_emission['Annual CO₂ emissions(tonnes)'] != 0]

df_country_emission.head()

#### Dataset 9: Read in data about CO2 emissions and awareness of climate change globally (for Yosef Analysis)¶

In [ ]:
df_global = pd.read_excel("Datasets/awareness_by_percentage.xlsx",header=1)
df_global.head()

Drop null values

In [ ]:
df_global.dropna

Keep less columns

In [ ]:
keep = ['Country','Population.2008','CO2emi','GDPperUS','glo_tot','Aware','Serious', 'WGI', 'Efcon']
df_global = df_global[keep]

Set country as the index

In [ ]:
df_global.set_index('Country')

Sort CO2emi and Aware columns

# Data Analysis & Results

Include cells that describe the steps in your data analysis.

#### Step \#1: First, we analyze the global CO2 emission

Graph CO2 Emissions and Awareness data.
Visualize the data in a choropleth map using geopandas.

In [ ]:
#sort CO2 emission 
df_global = df_global.sort_values('CO2emi')

In [ ]:
fig_size = plt.rcParams["figure.figsize"]

fig_size[0] = 12
fig_size[1] = 9
plt.rcParams["figure.figsize"] = fig_size
y_pos = np.arange(len(df_global['Country']))

plt.barh(y_pos, df_global['CO2emi'], align='center', alpha=0.5)
plt.yticks(y_pos, df_global['Country'])
plt.ylabel('CO2 Emissions in 2007-8')
plt.title('Country CO2 Emissions in 2007-8')
fig = plt.gcf()
fig.set_size_inches(18.5, 18.5)
plt.show()

In [ ]:
pip install folium

In [ ]:
pip install --upgrade earthpy

In [ ]:
import os 
import earthpy as et

import folium
from folium import plugins

import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, Resampling


In [ ]:
m = folium.Map(location=[37, -102], zoom_start=5)
m

In [ ]:
conda install geopandas

In [ ]:
!pip install descartes
!pip install PySAL
!pip install mapclassify

Import world geography dataset and merge with df_global.

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.head()

In [ ]:
country_shapes = world[['geometry', 'iso_a3']]

country_names = world[['name', 'iso_a3']]

# For spatial join
countries = world[['geometry', 'name']]

countries = countries.rename(columns={'name':'country'})
countries.head()

In [ ]:
from geopandas import GeoDataFrame

In [ ]:
df_map = df_global.merge(countries, left_on='Country', right_on='country')
df_geo = GeoDataFrame(df_map) 
print(df_geo)

In [ ]:
df_geo.plot()

In [ ]:
# plot choropleth for CO2emi
fig, ax = plt.subplots(1, 1, figsize=(17, 7))
divider = make_axes_locatable(ax)
df_geo.plot(column='CO2emi', ax=ax, cmap='GnBu', legend=True)
plt.title('CO2 Emissions')

# Discussion: An issue with this visualization: small, wealthy countries that have a lot of emmissions like Qatar and Luxemborg, are the highest but are not visible on this graph

In [ ]:
#sort Aware
df_global = df_global.sort_values('Aware')

In [ ]:
fig_size = plt.rcParams["figure.figsize"]

fig_size[0] = 12
fig_size[1] = 10
plt.rcParams["figure.figsize"] = fig_size
y_pos = np.arange(len(df_global['Country']))

plt.barh(y_pos, df_global['Aware'], align='center', alpha=0.5)
plt.yticks(y_pos, df_global['Country'])
plt.ylabel('Aware')
plt.title('Awareness in 2007-8')
fig = plt.gcf()
fig.set_size_inches(18.5, 32)
plt.show()

In [ ]:
# plot choropleth for awareness
fig, ax = plt.subplots(1, 1, figsize=(17, 7))
divider = make_axes_locatable(ax)
df_geo.plot(column='Aware', ax = ax, figsize=(17, 6), cmap='GnBu', legend = True);
plt.title('Awareness')

Explore the correlations in the global dataset

In [ ]:
df_global.corr()

Analysis: CO2emi is most strongly correlated with GDPperUS. Aware is most strongly correlated with glo_tot, GDPperUS, and WGI.

Discussion: I will focus on analyzing CO2emi and Aware, as per our goals for this project. Due to 
GDPperUS's strong correlation with these variables, I will examine it, as well. 
CO2emi is CO2 emissions per capita. Aware is the percent of people sampled who are aware that climate change is happening.
GDPperUS is GDP per capita in US dollars.
Although not examined here, for reference, glo_tot is globalization total, and WGI is World Governance Indicator.

Note: Some exploratory data analysis of the other variables in this dataset was excluded in order to keep the project coherent.

Plot scatter plots to visualize CO2emi and Aware

In [ ]:
plt.scatter(df_global['Aware'], df_global['CO2emi'])

plt.xlabel('Awareness')
plt.ylabel('CO2 Emissions')
#plt.legend('data');
#to do: look for clusters (maybe region, something else?)

#add line of best fit
b, m = polyfit(df_global['Aware'], df_global['CO2emi'], 1)
plt.plot(df_global['Aware'], b + m * df_global['Aware'], '-')

In [ ]:
plt.scatter(df_global['GDPperUS'], df_global['CO2emi'])

plt.xlabel('GDP per Capita in US dollars')
plt.ylabel('CO2 Emissions')

#add line of best fit
b, m = polyfit(df_global['GDPperUS'], df_global['CO2emi'], 1)
plt.plot(df_global['GDPperUS'], b + m * df_global['GDPperUS'], '-')

Analysis: CO2 emissions is positively correlates with awareness and with GDP. 

Discussion: The relationships are linear, so I do a linear regression to predict CO2 emissions from awareness.

In [ ]:
#linear model to predict CO2emi from awareness
outcome_1,predictors_1 = patsy.dmatrices('CO2emi ~ Aware', df_global)
mod_1 = sm.OLS(outcome_1,predictors_1)
res_1 = mod_1.fit()

print(res_1.summary())

Analysis: P is les than 0.01, so Aware predicts CO2emi.

Another scatter plot, with the model fit line

In [ ]:
## Plot the model fit line

# Plot the orginal data (as before)
plt.scatter(df_global['Aware'], df_global['CO2emi'], alpha=0.3, label='Data', s=100);

# Generate and plot the model fit line
xs = np.arange(df_global['CO2emi'].min(), df_global['CO2emi'].max())
ys = 0.1432 * xs - 4.2222
plt.plot(xs, ys, '--k', linewidth=4, label='Model')

plt.xlabel('Awareness')
plt.ylabel('CO2 Emissions')
plt.legend();

Discussion: While this model suggests that awareness predicts CO2 Emissions, GDP is correlated with both variables and may be a confounder. To explore this, I do a multivariate regression.

In [ ]:
#multivariate regression to predict CO2emi from both awareness and GDPperUS
outcome_2,predictors_2 = patsy.dmatrices('CO2emi ~ Aware + GDPperUS', df_global)
mod_2 = sm.OLS(outcome_2,predictors_2)
res_2 = mod_2.fit()

print(res_2.summary())

Analysis: P is less than 0.01, so awareness doesn't predict CO2 emissions when GDPperUS is taken into account. CO2 emissions is a confounding variable.

#### Step \#2: Then, we graph the annual greenhouse gas in US from 1990 to 2017

In [ ]:
#From dataset8, plot the diagram to show CO2 emission in US from 1800 to 2016
us_annuale = df_country_emission.loc[ 'United States' , : ]

lines = us_annuale.plot.line(x='Year', y='Annual CO₂ emissions(tonnes)', color='red', lw=8, figsize=(100,50))
plt.title('The annual CO₂ emission in US', size = 140)
plt.xlabel('Year', size = 70)
plt.ylabel('CO₂ emissions(tonnes)', size = 70)

In [ ]:
#Plot the diagram to show annual greenhouse gas in US over the recent year 
f = plt.figure(figsize = (18,8))
ax = f.add_subplot(1,1,1)
ax.plot(annual_gas_us['Year'], annual_gas_us['Total'])
ax.set_xlabel('Year', size = 15)
ax.set_ylabel('Total Emission (million metric tons)', size = 15)
ax.set_title('Annual Greenhouse Gas in US - 1990 to 2017', size = 25)

The diagram above shows the changes of annual greenhous gas in US from 1990 to 2017. We can easily notice that the changes are significant. The emission amount increased rapidly started from 1991 until 1999. And there is a peak in 2000 and the emission goes down again and keep constantly increasing until 2007. Starting from 2007, the emission amount got controlled and decreased rapidly within 2 years and keep under controlled.

#### Step \#3: Next, we graph the greenhouse gas by category

In [ ]:
#Generate barplot to show emission in US over year by category
f = plt.figure(figsize = (20,8))
ax = f.add_subplot(1,1,1)
bars = max(annual_gas_us['Total'])
r = annual_gas_us['Year']
barWidth = 1

plt.bar(r, annual_gas_us['Carbon dioxide'], edgecolor = 'white', width = barWidth, label = 'Carbon dioxide')
plt.bar(r, annual_gas_us['Methane'], edgecolor = 'white', width = barWidth, label = 'Methane')
plt.bar(r, annual_gas_us['Nitrous oxide'], edgecolor = 'white', width = barWidth, label = 'Nitrous oxide')
plt.bar(r, annual_gas_us['Fluorinated gases'], edgecolor = 'white', width = barWidth, label = 'Fluorinated gases')

plt.xlabel('Year', size = 15)
plt.ylabel('Proportion of Each Greenhouse Gas', size = 15)
plt.title('Greenhouse Gas in US by Category', size = 25)
plt.legend(loc = 5, prop = {'size': 14})

The barplot shows the proportion of each greenhouse gas from 1990 to 2017. And we can observe that the proportion of carbon dioxide are the highest among these four greenhous gas categories in every year.

### Hypothesis \#1: In this analysis, we're wondering whether the awareness of climate change affects their carbon dioxide emissions. We hypothesize that higher awareness of climate change will have lower carbon dioxide emissions. And vice versa.

we split our discussion of Hypothesis1 into two parts: First, in the level of US, we observe the awareness of climate change of US people from different states and compare it with carbon emission data. Second, we observe the CO2 emission in different countries all around the world, and then compare it with the global awareness of climate change.

##### 1. states level

In [ ]:
#sort the dataset in descending order
sort_state_2014 = df_YPCCC_2014_State_new.sort_values(by = ['happening'], ascending = False)

In [ ]:
#plot awareness states
sc = sns.color_palette("YlOrRd", 51)
sc.reverse()
sns.catplot(x = 'happening', y = 'Statename', kind = 'bar', palette = sc, data = sort_state_2014, height = 15)
plt.xlabel('happening', size = 20)
plt.ylabel('Statename', size = 20)
plt.title('Awareness of Climate Change in US - 2014', size = 20)

We can observe that the highest awareness of climate change state is Hawaii, and the lowest awareness of climate change state is West Virginia. Next, we will extract top 10 and least 10 states.

In [ ]:
#store top 10 awareness states
top10_aware_2014 = sort_state_2014.head(10)

#store least 10 awareness states
least10_aware_2014 = sort_state_2014.tail(10)

#### Next, we need both the same top10 and least10 awareness states in 2014 from the dataset 2016 in order to compare

In [ ]:
#extract top10 awareness states in 2014 from the dataset 2016
df_sametopstate_2016 = pd.DataFrame([])
for index, rows in df_YCOM_2016_state.iterrows():
    name = rows['GeoName']
    happen = rows['happening']
    for n in top10_aware_2014['Statename']:
        if name == n:
            df_sametopstate_2016 = df_sametopstate_2016.append({'Statename' : name, 'happening' : happen}, ignore_index = True)
            
#df_sametopstate_2016

In [ ]:
#extract least10 awareness states in 2014 from the dataset 2016
df_sameleaststate_2016 = pd.DataFrame([])
for index, rows in df_YCOM_2016_state.iterrows():
    name = rows['GeoName']
    happen = rows['happening']
    for n in least10_aware_2014['Statename']:
        if name == n:
            df_sameleaststate_2016 = df_sameleaststate_2016.append({'Statename' : name, 'happening' : happen}, ignore_index = True)
            
#df_sameleaststate_2016

#### Now, we are going to extract states's emission amount(both top10 and least10) in both 2014 and 2016 in order to compare the changes to see whether there is any correlation.

In [ ]:
#extract the top10 awareness state's correlated emission amount in 2014 and 2016
df_emissiontop10state = pd.DataFrame([])
for i, j in df_CO2_state_keep.iterrows():
    name = j['State']
    emission1 = j['2014']
    emission2 = j['2016']
    for n in top10_aware_2014['Statename']:
        if name == n:
            df_emissiontop10state = df_emissiontop10state.append({'Statename' : name, 'CO2_2014' : emission1,
                                                             'CO2_2016' : emission2}, ignore_index = True)
            
df_emissiontop10state = df_emissiontop10state[['Statename', 'CO2_2014', 'CO2_2016']]

#merge awareness based on statename
df_topstate = pd.merge(top10_aware_2014, df_sametopstate_2016, on = 'Statename')
df_topstate = pd.merge(df_topstate, df_emissiontop10state, on = 'Statename')
df_topstate = df_topstate[['Statename', 'happening_x', 'happening_y', 'CO2_2014', 'CO2_2016']]
df_topstate = df_topstate.rename(index = str, columns = {"happening_x": "Awareness2014", "happening_y": "Awareness2016"})
df_topstate

In [ ]:
#extract the least10 awareness state's correlated emission amount in 2014 and 2016
df_emissionleast10state = pd.DataFrame([])
for i, j in df_CO2_state_keep.iterrows():
    name = j['State']
    emission1 = j['2014']
    emission2 = j['2016']
    for n in least10_aware_2014['Statename']:
        if name == n:
            df_emissionleast10state = df_emissionleast10state.append({'Statename' : name, 'CO2_2014' : emission1,
                                                             'CO2_2016' : emission2}, ignore_index = True)
            
df_emissionleast10state = df_emissionleast10state[['Statename', 'CO2_2014', 'CO2_2016']]

#merge awareness based on statename
df_leaststate = pd.merge(least10_aware_2014, df_sameleaststate_2016, on = 'Statename')
df_leaststate = pd.merge(df_leaststate, df_emissionleast10state, on = 'Statename')
df_leaststate = df_leaststate[['Statename', 'happening_x', 'happening_y', 'CO2_2014', 'CO2_2016']]
df_leaststate = df_leaststate.rename(index = str, columns = {"happening_x": "Awareness2014", "happening_y": "Awareness2016"})
df_leaststate

## Relationship between awareness and emission. Will the awareness change over year affect the emission level?

In [ ]:
plt.rcParams["figure.figsize"] = (8,4)
df_topstate.plot(x = 'Statename', y = ['Awareness2014', 'CO2_2014'], kind = 'bar', color = ['black', 'red'])
plt.title('The top10 awareness states vs. CO2 in 2014', size = 15)
plt.xlabel('Statename', size = 15)
plt.xticks(rotation = 45)
plt.ylabel('Awareness/CO2', size = 15)

Based on the plot above, it plots the state based on the top awareness and their correlated CO2 emission in 2014. We have assumed that the highest awareness state will have lowest emission amount. However, according to the bar chart above, our assumption is not true. For example, we can observe that California's awareness is higher than Massachusetts, but the fact is that California's emission amount in 2014 is higher than Massachusetts.

In [ ]:
plt.rcParams["figure.figsize"] = (10,5)
df_leaststate.plot(x = 'Statename', y = ['Awareness2014', 'CO2_2014'], kind = 'bar', color = ['black', 'red'])
plt.title('The least10 awareness states vs. CO2 in 2014', size = 15)
plt.xlabel('Statename', size = 15)
plt.xticks(rotation = 45)
plt.ylabel('Awareness/CO2', size = 15)

Based on the plot above, it plots the state based on the least awareness in 2014. We have assumed that the lowest awareness state will have highest emission amount. However, according to the bar chart above, our assumption is not true. For example, we can observe that North Dakota's awareness is lower than Oklahoma, but the fact is that North Dakota's emission amount in 2014 is lower than Oklahoma.

## Now, let's look at the changes in awareness from 2014 to 2016

First, let's look at the top10 awareness states

In [ ]:
plt.rcParams["figure.figsize"] = (13,4)
plt.plot('Statename', 'Awareness2014', data = df_topstate, label = 'Awareness2014')
plt.plot('Statename', 'Awareness2016', data = df_topstate, label = 'Awareness2016')
plt.title('The top awareness state in 2014 compared to 2016', size = 15)
plt.xlabel('Statename', size = 15)
plt.ylabel('Awareness', size = 15)
plt.legend()

The plot shows that the awareness of climate change in the top10 states have changed in 2016. Overall, each of these states's awareness is increased.

#### Next, let's look at least10 awareness states too.

In [ ]:
plt.rcParams["figure.figsize"] = (13,4)
plt.plot('Statename', 'Awareness2014', data = df_leaststate, label = 'Awareness2014')
plt.plot('Statename', 'Awareness2016', data = df_leaststate, label = 'Awareness2016')
plt.title('The least awareness state in 2014 compared to 2016', size = 15)
plt.xlabel('Statename', size = 15)
plt.ylabel('Awareness', size = 15)
plt.legend()

The plot shows that the awareness of climate change in the least10 states have changed in 2016. Overall, each of these states's awareness is increased. Compared to the top awareness state plot above, the least awareness states have significant increased their awareness of climate change in two years.

#### Now, let's plot the data to see if the emission changed.

In [ ]:
plt.rcParams["figure.figsize"] = (10,3)
df_topstate.plot(x = 'Statename', y = ['CO2_2014', 'CO2_2016'], kind = 'bar')
plt.title('The top10 awareness states vs.their CO2 in 2014 and 2016', size = 15)
plt.xlabel('Statename', size = 15)
plt.xticks(rotation = 45)
plt.ylabel('CO2', size = 15)

By looking at this diagram, some of the states's CO2 emission is increased and some of the states's CO2 emission is decreased. Even though the fact that we obverse from the preivous histogram that the awareness level of all top10 awareness states in 2014 are increased, not all of their CO2 emission are decreased.

In [ ]:
plt.rcParams["figure.figsize"] = (10,3)
df_leaststate.plot(x = 'Statename', y = ['CO2_2014', 'CO2_2016'], kind = 'bar')
plt.title('The last10 awareness states vs. their CO2 in 2014 and 2016', size = 15)
plt.xlabel('Statename', size = 15)
plt.xticks(rotation = 45)
plt.ylabel('CO2', size = 15)

By looking at this diagram, some of the states's CO2 emission is increased and some of the states's CO2 emission is decreased. Same result as what we found from the top10 states in the previous diagram. Even though the fact that we obverse from the preivous histogram that the awareness level of all last10 awareness states in 2014 are increased, not all of their CO2 emission are decreased.

However, the interesting thing is that we can recognize that the CO2 emission amount has significant decreased within the last10 states compared to top10 states - which only decreased by a little amount.

##### 2. countries level

In [ ]:
sort_df_country_emission_2008 = df_country_emission_2008.sort_values(by = ['Annual CO₂ emissions(tonnes)'], ascending = False)
ax = sort_df_country_emission_2008.plot.bar(rot=0, figsize=(100,50))
sort_df_country_emission_2008 = sort_df_country_emission_2008.drop(['World'])
plt.title('Global CO2 Emmission Level in 2008', size = 100)
plt.xlabel('Countries', size = 80)
plt.ylabel('The Amount of Emission', size = 80)

In [ ]:
country_most_df_2008 = sort_df_country_emission_2008.head(10)
bx = country_most_df_2008.plot.bar(rot=0, figsize = (15,8), color = 'red')
plt.title('The top emission countries in 2008', size = 20)
plt.xlabel('Countries', size = 20)

In [ ]:
country_least_df_2008 = sort_df_country_emission_2008.tail(10)
cx = country_least_df_2008.plot.bar(rot=0, figsize=(18,8), color = 'green')
plt.title('The least emission countries in 2008', size = 20)
plt.xlabel('Countries', size = 20)

However, the 10 most and least CO2 emission by country level is not a fair rank and could not precisely quantify a countries responsibility for the climate change worldwide. By these statistics, it is clear that the top-ranked countries, like China, must contribute more on global warming than low-ranked countries, like Tuvalu, did in year 2008. But the population, the level of industrialization, and other confounds is worthy considering.

Thus, we read another data set which includes population and other confounds to analysis a country's CO2 emission level. Meanly, we compare different countries emission level by emission amount in a year per capita.

In [ ]:
df_country = pd.read_excel('Datasets/awareness_by_percentage.xlsx',header=1)

In [ ]:
#df_country = pd.read_excel('Datasets/awareness_by_percentage.xlsx',header=1)

#Extract the population
df_country_population = df_country.filter(items=['Country', 'Population.2008'])
df_country_population = df_country_population.rename(index=str, columns={"Country": "Entity", "Population.2008": "Population.2008"})
df_country_population.set_index('Entity', inplace=True)

#Merge it with the emission data and calculate the per 
sort_df_country_emission_2008 = sort_df_country_emission_2008.merge(df_country_population, on = 'Entity', validate = 'one_to_one')
sort_df_country_emission_2008['Emission/capita'] = sort_df_country_emission_2008['Annual CO₂ emissions(tonnes)'].div(sort_df_country_emission_2008['Population.2008'].values,axis=0)
sort_df_country_emission_2008

we also merge the awareness data in the data set for later comparison

In [ ]:
# Extract the awareness data
df_country_aware = df_country.filter(items=['Country', 'Aware'])
df_country_aware = df_country_aware.rename(index=str, columns={"Country": "Entity", "Population.2008": "Population.2008"})
df_country_aware.set_index('Entity', inplace=True)

# Merge it
country_emission_awareness_2008 = sort_df_country_emission_2008.merge(df_country_aware, on = 'Entity', validate = 'one_to_one')
sort_country_emission_awareness_2008 = country_emission_awareness_2008.sort_values(by = ['Emission/capita'], ascending = False)
sort_country_emission_awareness_2008 = sort_country_emission_awareness_2008.drop(columns = ['Annual CO₂ emissions(tonnes)','Population.2008'])

In [ ]:
#plot the most emission countries
df_most_cemission = sort_country_emission_awareness_2008['Emission/capita'].head(10)
dx = df_most_cemission.plot.bar(rot=0, figsize=(20,8), color = 'red')
plt.title('The top emission/capita countries in 2008', size = 20)
plt.xlabel('Countries', size = 20)

In [ ]:
#plot the corresponding awareness data
df_most_cemission_awareness = sort_country_emission_awareness_2008['Aware'].head(10)
ex = df_most_cemission_awareness.plot.bar(rot=0, figsize=(20,8), color = 'purple')
plt.title('The awareness of the top emission/capita countries in 2008', size = 20)
plt.xlabel('Countries', size = 20)

In [ ]:
#Plot the scatter plot for these data
most_emission_awareness = sort_country_emission_awareness_2008.head(10)
most_emission_awareness = most_emission_awareness.reset_index()

x = most_emission_awareness['Emission/capita']
y = most_emission_awareness['Aware']
types = most_emission_awareness['Entity']

for i,j in enumerate(types):
    x_coor = x[i]
    y_coor = y[i]
    plt.plot(x_coor, y_coor, marker = '.', color = 'red', markersize = 10)
    plt.text(x_coor + 0.5, y_coor + 0.5, j, fontsize = 13)
    plt.xlabel('2008 year emission', fontsize = 20)
    plt.ylabel('Awareness', fontsize = 20)
    
plt.gcf().set_size_inches((20, 15))
plt.show()

In [ ]:
least_emission_awareness = country_emission_awareness_2008.tail(10).drop(columns = ['Annual CO₂ emissions(tonnes)','Population.2008'])
least_emission_awareness = least_emission_awareness.sort_values(by = ['Emission/capita'], ascending = True)
df_least_cemission = least_emission_awareness['Emission/capita'].tail(10)

ex = df_least_cemission.plot.bar(rot=0, figsize = (19,8), color = 'green')
plt.title('The least emission/capita countries in 2008', size = 20)
plt.xlabel('Countries', size = 20)

In [ ]:
df_least_cemission_awareness = least_emission_awareness['Aware']
fx = df_least_cemission_awareness.plot.bar(rot=0, figsize = (19,8), color = 'orange')
plt.title('The awareness of least emission/capita countries in 2008', size = 20)
plt.xlabel('Countries', size = 20)

In [ ]:
least_emission_awareness = least_emission_awareness.reset_index()

x = least_emission_awareness['Emission/capita']
y = least_emission_awareness['Aware']
types = least_emission_awareness['Entity']

for i,j in enumerate(types):
    x_coor = x[i]
    y_coor = y[i]
    plt.plot(x_coor, y_coor, marker = '.', color = 'green', markersize = 20)
    plt.text(x_coor+0.05, y_coor+0.05, j, fontsize = 13)
    plt.xlabel('2008 year emission', fontsize = 20)
    plt.ylabel('Awareness', fontsize = 20)
    
plt.gcf().set_size_inches((20, 15))
plt.show()

Suprisingly, when comparing the awareness of the top/least emission country, we find that the citizens in the top 10 emission countries have a higher level of being aware of the climate change than the citizens in the least emission countries. Thus, it doesn't fit in our hypothesis.

### Hypothesis \#2: In this analysis, we're wondering whether a state's political party affects their carbon dioxide emissions. We hypothesize that Democratic states are more likely to believe that climate change is happening, and that those states would thus have smaller amounts of carbon dioxide emissions. Our reasoning for this is that Republican politicians tend to outwardly deny climate change more than Democratic politicians, and there has been a recent wave among Democratic presidential candidates to deny fossil fuel money. 

#### Step \#1: First, we graph each state's CO2 emissions. 

In [ ]:
#Create a dictionary to store state-CO2 key-value pairs: 

states = df_party_state.index.values
CO2 = df_CO2_state_2014['CO2_Emissions_in_2014']
keys = states
values = CO2

CO2_states_dict = dict(zip(keys, values))
sorted_CO2_states_dict = sorted(CO2_states_dict.items(), key=operator.itemgetter(1))
#sorted_CO2_states_dict

In [ ]:
#Plot US CO2 Emissions in 2014 by State
fig_size = plt.rcParams["figure.figsize"]

fig_size[0] = 12
fig_size[1] = 9
plt.rcParams["figure.figsize"] = fig_size

States = []
CO2 = []

for state, co2 in sorted_CO2_states_dict: 
    States.append(state)
    CO2.append(co2)

y_pos = np.arange(len(States))

plt.barh(y_pos, CO2, align='center', alpha=0.5)
plt.yticks(y_pos, States)
plt.ylabel('CO2 Emissions (million metric tonnes) in 2014')
plt.title('U.S. State CO2 Emissions in 2014')

plt.show()

##### Discussion: From the above graph, we see that the state of Texas had the highest rate of CO2 emissions in 2014, with approximately 626 million metric tonnes of CO2 emissions. The second highest CO2 emissions in 2014 came from California, and the lowest amount of emissions came from the state of Vermont. 

#### Step \#2: Next, we overlay each state's political affiliation onto their CO2 emissions on the graph:

In [ ]:
#Now color code the graph based on the political affiliation of each state: 
state_party_dict = {}
for state in df_party_state.index.values:
    if float(df_party_state.at[state, 'Republican']) > float(df_party_state.at[state, 'Democrat']):
        state_party_dict[state] = 'Republican'
    else:
        state_party_dict[state] = 'Democrat'
        
#print(state_party_dict)
#print(sorted_CO2_states_dict)
        
colors = []
sorted_states_by_CO2 = [state[0] for state in sorted_CO2_states_dict]

for state in sorted_states_by_CO2: 
    if state_party_dict[state] == 'Republican':
        colors.append('red')
    else:
        colors.append('blue')

#print(colors)

In [ ]:


fig_size = plt.rcParams["figure.figsize"]

fig_size[0] = 12
fig_size[1] = 9
plt.rcParams["figure.figsize"] = fig_size

States = []
CO2 = []

for state, co2 in sorted_CO2_states_dict: 
    if state != "District of Columbia":
        States.append(state)
        CO2.append(co2)

y_pos = np.arange(len(States))

plt.barh(y_pos, CO2, align='center', alpha=0.5, color=colors)
plt.yticks(y_pos, States)
plt.ylabel('CO2 Emissions (million metric tonnes) in 2014')
plt.title('U.S. State CO2 Emissions in 2014')

blue_patch = mpatches.Patch(color='blue',alpha=0.5, label='Democrat')
red_patch = mpatches.Patch(color='red', alpha=0.5, label='Republican')

plt.legend(handles=[blue_patch, red_patch])

plt.show()

#### Discussion: In the above graph, blue denotes majority Democratic states while red denotes Republican states in 2014. Here, we see that both the highest 4 and lowest 4  emitters of CO2 are Democratic states, which shows us that there doesn't seem to be a clear correlation between a state's CO2 emissions and their political party affiliation. However, we notice that the states with the largest amounts of CO2 emissions are also the largest states in terms of land area. Therefore, we will perform an analysis that looks at the amount of CO2 released per size of the state, by dividing CO2 emissions by the state's size in sq. miles.

#### Step \#3: Next, we divide each state's CO2 emissions by the size of each state, to get a better idea of the CO2 emission density (since larger states on average emit more CO2): 

In [ ]:
#Import a new dataset with state size in sq. miles: 
df_state_size = pd.read_csv("Datasets/2014_State_Size_Population_Density.csv")
df_state_area = df_state_size.drop(["Population", "Density (people per sq mi)"], axis = 1)

##Remove the rankings column
new_land_area = []
for x in df_state_area["Land Area (sq mi)"]:
    text = x.split()
    new_land_area.append(text[0])
      
df_state_area["Land Area (sq mi)"] = new_land_area
df_state_area


##Remove territories that aren't in previous dataset, since they aren't states:
list_not_states = []
for s,a in zip(df_state_area['State'], df_state_area["Land Area (sq mi)"]):
    if s not in States:
        #print(s)
        df_state_area = df_state_area[df_state_area.State != s]

#df_state_area


In [ ]:
#Divide CO2 emissions by state size: 
states = df_state_area['State']
areas = df_state_area['Land Area (sq mi)']
states_area_dict = dict(zip(states, areas))

CO2_div_area_states_dict = CO2_states_dict

for state in States: 
    state_area = states_area_dict[state]
    state_area = float(state_area.replace(',', ''))
    CO2_div_area_states_dict[state] = float(CO2_div_area_states_dict[state]/state_area)
    
#CO2_div_area_states_dict

In [ ]:
#Sort dictionary by CO2 emissions/state size:
sorted_CO2_div_area_states_dict = sorted(CO2_div_area_states_dict.items(), key=operator.itemgetter(1))
sorted_CO2_div_area_states_dict

#print(sorted_CO2_states_dict)
        
colors = []
sorted_states_by_CO2_div_area = [state[0] for state in sorted_CO2_div_area_states_dict]

for state in sorted_states_by_CO2_div_area: 
    if state_party_dict[state] == 'Republican':
        colors.append('red')
    else:
        colors.append('blue')

#print(colors)

In [ ]:
fig_size = plt.rcParams["figure.figsize"]

fig_size[0] = 12
fig_size[1] = 9
plt.rcParams["figure.figsize"] = fig_size

States = []
CO2 = []

for state, co2 in sorted_CO2_div_area_states_dict: 
    if state != "District of Columbia":
        States.append(state)
        CO2.append(co2)
        #CO2.append(math.log(co2*(10**16)))

y_pos = np.arange(len(States))

plt.barh(y_pos, CO2, align='center', alpha=0.5, color=colors)
plt.yticks(y_pos, States)
plt.ylabel('CO2 Emissions (million metric tonnes) divided by state area size (sq miles) in 2014')
plt.title('U.S. State CO2 Emissions in 2014')

blue_patch = mpatches.Patch(color='blue', alpha=0.5, label='Democrat')
red_patch = mpatches.Patch(color='red', alpha=0.5, label='Republican')

plt.legend(handles=[blue_patch, red_patch])

plt.show()

#### Discussion: Here, it seems to be the case that Democratic states are actually releasing more CO2 than Republican states. Let's perform a t-test to analyze whether there is a statistical correlation between a state's political party and their CO2 emissions: 

In [ ]:
#To quantify correlation, look at mean and histogram of democratic vs. republican states:
democrat_states = []
democrat_CO2 = []
republican_states = []
republican_CO2 = []
index = 0

for state, CO2 in sorted_CO2_div_area_states_dict: 
    if colors[index] == 'blue':
        democrat_states.append(state)
        democrat_CO2.append(float(CO2))
    else:
        republican_states.append(state)
        republican_CO2.append(float(CO2))
    index = index + 1
        
democrat_CO2 = np.array(democrat_CO2)
republican_CO2 = np.array(republican_CO2)

print("democrat_CO2 mean:", democrat_CO2.mean())
print("republican_CO2 mean:", republican_CO2.mean())

print("\n")

print("democrat_CO2 variance:", democrat_CO2.var())
print("republican_CO2 variance:", republican_CO2.var())


In [ ]:
from scipy import stats
#Perform t-test:

t, two_p =stats.ttest_ind(democrat_CO2, republican_CO2, equal_var = True)
print("\n Statistics: \n")
print("\t t-statistic: ", t)
print("\t two-tailed p-value: ", two_p)

print("\n Results: \n")
if two_p < 0.05: 
    print("\t We reject the null hypothesis that the distributions are the same. A significant difference between \n the two populations was found!")
else:
    print("\t We do not reject the null hypothesis. The two distributions are the same. No significant difference was found.")

#### Analysis: After doing our t-test, we get a two-tailed p-value of 0.02. Since this is less than the critical value of alpha = 0.05,  we reject the null hypothesis that the two groups' means were the same. This means that Democratic states did signficantly release more carbon dioxide emissions than Republican states in 2014, according to these datasets. 

#### Now, we wonder: Does a state's political party affiliation has any correlation to the state population's awareness and perception of climate change?


#### Step \#4: Load in a dataset of each state's perception of climate change:

In [ ]:
df_YPCCC_2014_State_new = df_YPCCC_2014_State_new[df_YPCCC_2014_State_new.Statename != "District of Columbia"]
df_YPCCC_2014_State_new

In [ ]:
states = df_YPCCC_2014_State_new["Statename"]

happenings = df_YPCCC_2014_State_new["happening"]
states_happening_dict = dict(zip(states, happenings))

sorted_states_happening_dict = sorted(states_happening_dict.items(), key=operator.itemgetter(1))
#print(sorted_states_happening_dict)
 
sorted_states_happening_colors = []

for state, happening in sorted_states_happening_dict: 
    if state_party_dict[state] == 'Republican':
        sorted_states_happening_colors.append('red')
    else:
        sorted_states_happening_colors.append('blue')

#print(sorted_states_happening_colors)



In [ ]:

fig_size = plt.rcParams["figure.figsize"]

fig_size[0] = 12
fig_size[1] = 9
plt.rcParams["figure.figsize"] = fig_size

States = []
Happening = []

for state, hap in sorted_states_happening_dict: 
    if state != "District of Columbia":
        States.append(state)
        Happening.append(hap)

y_pos = np.arange(len(States))

plt.barh(y_pos, Happening, align='center', alpha=0.5, color=sorted_states_happening_colors)
plt.yticks(y_pos, States)
plt.xlabel('% of State Population that believes Global Warming is happening')
plt.ylabel('State')

plt.title('Correlation between State Political Party and whether that state believes Global Warming is happening')

blue_patch = mpatches.Patch(color='blue', alpha=0.5, label='Democrat')
red_patch = mpatches.Patch(color='red', alpha=0.5, label='Republican')

plt.legend(handles=[blue_patch, red_patch])

plt.show()

#### Discussion: Here, it seems to be the case that Democratic states are more aware that global warming is happening, compared to Republican states. Again, let's perform a t-test to analyze whether there is a statistical correlation between a state's political party and their CO2 emissions: 

In [ ]:
#To quantify correlation, look at mean and histogram of democratic vs. republican states:
democrat_states = []
democrat_happening = []
republican_states = []
republican_happening = []
index = 0

for state, hap in sorted_states_happening_dict: 
    if sorted_states_happening_colors[index] == 'blue':
        democrat_states.append(state)
        democrat_happening.append(float(hap))
    else:
        republican_states.append(state)
        republican_happening.append(float(hap))
    index = index + 1
        
democrat_happening = np.array(democrat_happening)
republican_happening = np.array(republican_happening)

print("democrat_happening mean:", democrat_happening.mean())
print("republican_happening mean:", republican_happening.mean())

print("\n")

print("democrat_happening variance:", democrat_happening.var())
print("republican_happening variance:", republican_happening.var())


In [ ]:
#Perform t-test:

t, two_p =stats.ttest_ind(democrat_happening, republican_happening, equal_var = False)
print("\n Statistics: \n")
print("\t t-statistic: ", t)
print("\t two-tailed p-value: ", two_p)

print("\n Results: \n")
if two_p < 0.05: 
    print("\t We reject the null hypothesis that the distributions are the same. A significant difference between \n the two populations was found!")
else:
    print("\t We do not reject the null hypothesis. The two distributions are the same. No significant difference was found.")

#### Analysis: Here, we again reject the null hypothesis that the two distributions are the same, which means that there was a statistical correlation between political affiliation and whether or not a population believed global warming was happening in 2014, based on these datasets. From this result, we can say that overall, democratic states are more likely  than populations in republican states to believe that global warming is happening. 


### Conclusion of Analysess: 

#### While Democratic states appear to emit more CO2 than Republican states in 2014, they also appear to be more aware that climate change is happening, which seems contradictory. This causes us to believe that there may be compounding variables that we have not considered in our analysis. What could explain why states who are more aware that global warming is happening still release more carbon dioxide? For future work, we may want to look at a state's GDP as a potential compounding factor. 

# Ethics & Privacy

Common ethical considerations in research include concerns of privacy, consent and
beneficence. Privacy and authorship are ensured in our study as all of the datasets we have
decided to use are publicly available and have been anonymized. These data are mainly about
national carbon emissions and anonymous individual opinions on climate change, which are then
grouped based on their geographic location. The final aima of this research is to promote an
awareness of climate change, which we believe would be advantageous to anyone who could
potentially be affected by this research.

One ethical concern we do have is that in being American and doing this study at a
university in the United States, we inherently bring a western perspective of climate change to the
analysis of other countries. We will need to be mindful of any inherent bias we have with this
perspective, and actively work to mitigate it by understanding that there are geographic and
cultural differences which shape an individual’s perception, awareness of, and response to climate
change. We may be able to avoid this bias by doing qualitative research of climate change
perceptions in various non-Western countries.

# Conclusion & Discussion

*Fill in your discussion information here*